In [ ]:
import  numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeCV

from sklearn.preprocessing import QuantileTransformer, quantile_transform

from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error
# ....

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
# import lightgbm as lgb 

from lightgbm import LGBMRegressor,LGBMClassifier
from sklearn.pipeline import Pipeline, make_pipeline

from scipy.stats import uniform

from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from datetime import datetime

# tuner.get_best_hyperparameters(num_trials=1)[0].values
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from datetime import datetime
!pip3 install pickle5
import pickle5 as pickle
from sklearn.utils import resample

from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RepeatedKFold,RepeatedStratifiedKFold
import joblib



%matplotlib inline

In [ ]:
# Load cleaned dummy variable data
with open('cleaned_data_dummy_vars.pickle', 'rb') as handle:
    background_imputed_tot = pickle.load(handle)
    
with open('X_train_dummy_vars.pickle', 'rb') as handle:
    X_train = pickle.load(handle)

with open('X_CV_dummy_vars.pickle', 'rb') as handle:
    X_CV = pickle.load(handle)

with open('x_test_dummy_vars.pickle', 'rb') as handle:
    x_test = pickle.load(handle)
    
with open('x_leaderboard_dummy_vars.pickle', 'rb') as handle:
    x_leaderboard = pickle.load(handle)
    
with open('y_train_dummy_vars.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
    
with open('y_CV_dummy_vars.pickle', 'rb') as handle:
    y_CV = pickle.load(handle)
    
background = pd.read_csv('FFChallenge_v5/background.csv', sep=',', header=0,index_col=0,low_memory=False)
# train.csv contains 2,121 rows (one per child in the training set) and 7 columns.
train = pd.read_csv('FFChallenge_v5/train.csv', sep=',', header=0, index_col=0,low_memory=False)
########### Holdout dataset for internal testing only
test = pd.read_csv('test.csv',header=0, index_col=0,low_memory=False)
leaderboard = pd.read_csv('leaderboard.csv', header=0, index_col=0,low_memory=False)
leaderboard = leaderboard.dropna()

In [ ]:
# Final stats and useful variables associated with each column

numerical_columns = [c for c,v in background_imputed_tot.dtypes.iteritems() if v in [np.float,np.int,np.int64]]
non_numerical_columns = [c for c,v in background_imputed_tot.dtypes.iteritems() if v not in [np.float,np.int,np.int64]]

continuous_cols_lst = list()
continuous_cols_lst = background_imputed_tot.T.loc[(background_imputed_tot.apply(pd.Series.nunique) >= 15).values==True].index.to_list()
print('Number of continuous columns %s' % len(continuous_cols_lst))


Number of continuous columns 507


## Training elastic net on continuous variables

In [ ]:
best_params_elastic_net = dict() # saving best hyperparameters
for variable in ['gpa','grit','materialHardship']:
    best_p_dict = dict()
    print('---------------variable:',variable,'---------------\n\n\n')
    # Get non-na rows for this specific variable
    y_train_no_na = y_train[variable].dropna()
    X_train_no_na = X_train.loc[y_train_no_na.index.values]

    y_CV_no_na = y_CV.copy()[variable].dropna()
    X_CV_no_na = X_CV.copy().loc[y_CV_no_na.index.values]

    mask = leaderboard[variable].isna()
    x_leaderboard = background_imputed_tot.loc[leaderboard.index.values]
    y_leaderboard = leaderboard[variable]
    x_leaderboard_no_na = x_leaderboard.loc[y_leaderboard.index]

    # y_train_trans= y_train_no_na.copy()

    # Standardizing outcome to be between 0 and 1 and apply the same transformation on the validation and test sets
    y_train_mean = y_train_no_na.mean()
    y_train_std = y_train_no_na.std()


    y_train_trans = (y_train_no_na-y_train_mean)/y_train_std
    # y_CV_trans = y_CV_no_na.copy()
    # y_CV_trans = (y_CV_no_na-y_train_mean)/y_train_std
    y_CV_trans = y_CV_no_na.copy()

    # y_leaderboard = (y_leaderboard-y_train_mean)/y_train_std

    y_test_trans = test[variable].dropna()
    # y_test_trans = (y_test_trans-y_train_mean)/y_train_std
    x_test_no_na = x_test.loc[y_test_trans.index.values]



    ##### Standardizing features to be between 0 and 1 and apply the same transformation on the validation and test sets
    # Pandas applies these operations row-wise
    X_train_normalized = X_train_no_na.copy()
    X_cv_normalized = X_CV_no_na.copy()
    x_leaderboard_normalized = x_leaderboard_no_na.copy()




    ##### Normalize features to be between 0 and 1 and apply the same transformation on the validation and test sets
    # Pandas applies these operations row-wise
    X_train_standardized = X_train_no_na.copy()
    X_cv_standardized = X_CV_no_na.copy()
    x_leaderboard_standardized = x_leaderboard_no_na.copy()
    x_test_standardized = x_test_no_na.copy()

    # Standardising continuous columns to 0 mean, and std of 1
    column_mean_normalization = X_train_no_na[continuous_cols_lst].mean()
    column_std_normalization = X_train_no_na[continuous_cols_lst].std()


    X_train_standardized[continuous_cols_lst]= (X_train_no_na[continuous_cols_lst]-column_mean_normalization)\
    /(column_std_normalization)

    X_cv_standardized[continuous_cols_lst]=(X_CV_no_na[continuous_cols_lst]-column_mean_normalization)\
    /(column_std_normalization)

    x_leaderboard_standardized[continuous_cols_lst] = (x_leaderboard_no_na[continuous_cols_lst]-column_mean_normalization)\
    /(column_std_normalization)

    x_test_standardized[continuous_cols_lst] = (x_test_no_na[continuous_cols_lst]-column_mean_normalization)\
    /(column_std_normalization)


    parametersGrid = {"max_iter":[10000],
                    "tol":[0.005],
                      "alpha": [0.001,0.003, 0.01,0.03,0.1,0.3, 1,3, 10,30, 100],
                    "l1_ratio": np.arange(0.2, 1.0, 0.2)}

    eNet = ElasticNet()
    grid = GridSearchCV(eNet, parametersGrid, scoring='neg_mean_squared_error', cv=RepeatedKFold(n_splits=3, n_repeats=2),verbose=1)
    grid.fit(X_train_standardized, y_train_no_na)

    alpha_val = grid.best_params_['alpha']
    l1_ratio_val = grid.best_params_['l1_ratio']

    # Only saving alpha and l1 value, since the other parameters are fixed
    best_p_dict['alpha'] = alpha_val
    best_p_dict['l1_ratio'] = l1_ratio_val
    best_params_elastic_net[variable] = best_p_dict # Adding best parameters to the dictionnary of values

    model = ElasticNet(alpha=alpha_val,l1_ratio=l1_ratio_val,max_iter=10000,tol=0.005)
    model.fit(X_train_standardized, y_train_trans)

    # Save best model
    joblib.dump(model, 'elastic_net_'+ variable + '.pkl')


    coefs = model.coef_
    print('num coefs > 0 =', len(coefs[coefs> 0]))


    y_pred_train = model.predict(X_train_standardized)
    y_pred_all = model.predict(x_leaderboard_standardized)
    cv_preds = model.predict(X_cv_standardized)
    test_preds = model.predict(x_test_standardized)

    # Now remove normalization to accurate comparison to baseline
    y_pred_train = (y_pred_train*y_train_std)+y_train_mean
    y_pred_all = (y_pred_all*y_train_std)+y_train_mean
    cv_preds = (cv_preds*y_train_std)+y_train_mean
    test_preds = (test_preds*y_train_std)+y_train_mean


    # Can use mean square error as performance metric for both, since for 1 output, the mean square error and brier loss are the same.
    print('---- Train error ----')
    print(mean_squared_error(y_train_no_na,y_pred_train))
    print('---- CV error ----')
    print(len(y_CV_trans))
    print(len(cv_preds))
    print(mean_squared_error(y_CV_trans,cv_preds))
    # train_cv_errors[variable] = [mean_squared_error(y_train_trans,y_pred_train),mean_squared_error(y_CV_trans,cv_preds)]

    # y_CV_trans = (y_CV_no_na-y_train_mean)/y_train_std
    print('---- leaderboard stats ----')
    print(r2_score(y_leaderboard[~mask],y_pred_all[~mask]))
    print(mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask]))
    print('\n\n\n')
    preds_baseline_test = np.array([y_test_trans.mean()for i in range(len(y_test_trans))])
    print('Baseline test MSE =',mean_squared_error(y_test_trans,preds_baseline_test))

    print('Model test MSE =',mean_squared_error(test_preds,y_test_trans))
    # acc_nns[variable] = mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask])
    print('\n\n\n\n')


---------------variable: gpa ---------------



Fitting 6 folds for each of 44 candidates, totalling 264 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 264 out of 264 | elapsed:  5.5min finished


num coefs > 0 = 49
---- Train error ----
0.328690298399009
---- CV error ----
240
240
0.3910418899712505
---- leaderboard stats ----
0.003792910535217686
0.3891345281145492




Baseline test MSE = 0.42235637187960196
Model test MSE = 0.3633380639614692





---------------variable: grit ---------------



Fitting 6 folds for each of 44 candidates, totalling 264 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 264 out of 264 | elapsed:  6.7min finished


num coefs > 0 = 32
---- Train error ----
0.21564021892159743
---- CV error ----
281
281
0.20996893990889384
---- leaderboard stats ----
0.009504623956038571
0.21764914959324488




Baseline test MSE = 0.25292320173066524
Model test MSE = 0.24609645049709947





---------------variable: materialHardship ---------------



Fitting 6 folds for each of 44 candidates, totalling 264 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 264 out of 264 | elapsed:  4.2min finished


num coefs > 0 = 574
---- Train error ----
0.004149263453137513
---- CV error ----
288
288
0.02220667097532756
---- leaderboard stats ----
0.03239461089498097
0.02767916394247585




Baseline test MSE = 0.024827526011157196
Model test MSE = 0.023315472416012787







## Training logistic regression with elastic net regularization on categorical outcomes

In [ ]:
for variable in ['eviction','jobTraining','layoff']:
    best_p_dict = dict()
    print('---------------variable:',variable,'---------------\n\n\n')
    # Get non-na rows for this specific variable

    y_train_no_na = y_train[variable].dropna()
    X_train_no_na = X_train.loc[y_train_no_na.index.values]

    mask = leaderboard[variable].isna()
    x_leaderboard = background_imputed_tot.loc[leaderboard.index.values]
    y_leaderboard = leaderboard[variable]
    x_leaderboard_no_na = x_leaderboard.loc[y_leaderboard.index]

    y_train_trans= y_train_no_na.copy()

    y_CV_trans = y_CV_no_na.copy()

    # y_leaderboard = (y_leaderboard-y_train_mean)/y_train_std

    y_test_trans = test[variable].dropna()
    # y_test_trans = (y_test_trans-y_train_mean)/y_train_std
    x_test_no_na = x_test.loc[y_test_trans.index.values]



    ##### Standardizing features to be between 0 and 1 and apply the same transformation on the validation and test sets
    # Pandas applies these operations row-wise
    X_train_normalized = X_train_no_na.copy()
    X_cv_normalized = X_CV_no_na.copy()
    x_leaderboard_normalized = x_leaderboard_no_na.copy()




    ##### Normalize features to be between 0 and 1 and apply the same transformation on the validation and test sets
    # Pandas applies these operations row-wise
    X_train_standardized = X_train_no_na.copy()
    X_cv_standardized = X_CV_no_na.copy()
    x_leaderboard_standardized = x_leaderboard_no_na.copy()
    x_test_standardized = x_test_no_na.copy()

    # Standardising continuous columns to 0 mean, and std of 1
    column_mean_normalization = X_train_no_na[continuous_cols_lst].mean()
    column_std_normalization = X_train_no_na[continuous_cols_lst].std()


    X_train_standardized[continuous_cols_lst]= (X_train_no_na[continuous_cols_lst]-column_mean_normalization)\
    /(column_std_normalization)

    X_cv_standardized[continuous_cols_lst]=(X_CV_no_na[continuous_cols_lst]-column_mean_normalization)\
    /(column_std_normalization)

    x_leaderboard_standardized[continuous_cols_lst] = (x_leaderboard_no_na[continuous_cols_lst]-column_mean_normalization)\
    /(column_std_normalization)

    x_test_standardized[continuous_cols_lst] = (x_test_no_na[continuous_cols_lst]-column_mean_normalization)\
    /(column_std_normalization)

    # weights = y_train_trans.value_counts().to_dict()
    # temp = weights[0]
    # weights[0] = weights[1]
    # weights[1] = temp

    parametersGrid = {"C": [0.03,0.1,0.3, 1,3, 10,30], # Note: After testing values heiristically, values of C of less than 0.03 lead to models with 0 parameters remaining
                    "l1_ratio": np.arange(0.2, 1.0, 0.2),
                    "max_iter":[10000],
                    "solver" : ["saga"],
                    "penalty" : ["elasticnet"],
                    "tol" : [0.01]}

    eNet = LogisticRegression()
    grid = GridSearchCV(eNet, parametersGrid, scoring='neg_mean_squared_error', cv=RepeatedStratifiedKFold(3,2),verbose = 1)
    grid.fit(X_train_standardized, y_train_no_na)

    # train model using best parameters
    C_val = grid.best_params_['C']
    l1_ratio_val = grid.best_params_['l1_ratio']
    best_p_dict['C'] = C_val
    best_p_dict['l1_ratio'] = l1_ratio_val
    best_params_elastic_net[variable] = best_p_dict # Adding best parameters to the dictionnary of values


    model = LogisticRegression(penalty = 'elasticnet', solver = 'saga',C = 0.02, l1_ratio = l1_ratio_val,max_iter=1000,tol = 0.01)
    model.fit(X_train_standardized, y_train_trans)

    joblib.dump(model, 'elastic_net_'+ variable + '.pkl')

    # coefs = model.coef_
    # print('num coefs > 0 =', len(coefs[coefs> 0]))


    y_pred_train = model.predict_proba(X_train_standardized)[:,1]
    y_pred_all = model.predict_proba(x_leaderboard_standardized)[:,1]
    cv_preds = model.predict_proba(X_cv_standardized)[:,1]
    test_preds = model.predict_proba(x_test_standardized)[:,1]


    # Can use mean square error as performance metric for both, since for 1 output, the mean square error and brier loss are the same.
    print('---- Train error ----')
    print(mean_squared_error(y_train_no_na,y_pred_train))
    print('---- CV error ----')
    # print(len(y_CV_trans))
    # print(len(cv_preds))
    print(mean_squared_error(y_CV_trans,cv_preds))
    # train_cv_errors[variable] = [mean_squared_error(y_train_trans,y_pred_train),mean_squared_error(y_CV_trans,cv_preds)]

    # y_CV_trans = (y_CV_no_na-y_train_mean)/y_train_std
    print('---- leaderboard error ----')
    print(mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask]))
    print('\n\n\n')
    preds_baseline_test = np.array([y_test_trans.mean()for i in range(len(y_test_trans))])
    print('Baseline test brier loss =',mean_squared_error(y_test_trans,preds_baseline_test))

    print('Model test brier loss =',mean_squared_error(test_preds,y_test_trans))
    # acc_nns[variable] = mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask])
    print('\n\n\n\n')

# Save best parameters to pickle file
with open('best_params_elastic_net.pickle', 'wb') as handle:
    pickle.dump(best_params_elastic_net, handle, protocol=pickle.HIGHEST_PROTOCOL)




---------------variable: eviction ---------------



Fitting 6 folds for each of 28 candidates, totalling 168 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 168 out of 168 | elapsed: 38.3min finished


---- Train error ----
0.049183498870090915
---- CV error ----
0.018214543052421192
---- leaderboard error ----
0.05228921042504571




Baseline test brier loss = 0.0554574230504624
Model test brier loss = 0.05339300604782739





---------------variable: jobTraining ---------------



Fitting 6 folds for each of 28 candidates, totalling 168 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 168 out of 168 | elapsed: 35.9min finished


---- Train error ----
0.1606664346772303
---- CV error ----
0.044938240986095196
---- leaderboard error ----
0.20188079209608278




Baseline test brier loss = 0.18521499553665194
Model test brier loss = 0.17827193865246235





---------------variable: layoff ---------------



Fitting 6 folds for each of 28 candidates, totalling 168 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 168 out of 168 | elapsed: 28.6min finished


---- Train error ----
0.15483090857016998
---- CV error ----
0.0380161469958675
---- leaderboard error ----
0.17493812539951997




Baseline test brier loss = 0.16721354282637468
Model test brier loss = 0.16588094171131537





